In [1]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
import evaluate
from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation

2023-10-19 10:36:05.098018: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-19 10:36:05.116920: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-19 10:36:05.116936: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-19 10:36:05.116956: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-19 10:36:05.120850: I tensorflow/core/platform/cpu_feature_g

In [2]:
def jpeg_compress_ade(sample):
    img,bpp = jpeg_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample
    
def webp_compress_ade(sample):
    img,bpp = webp_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to("cpu")
def mbt2018_compress_ade(sample):
    img,bpp = nn_compress(sample['image'],net_mbt2018,"cpu")
    sample['image'] = img
    sample['bpp'] = bpp
    return sample

def hific_lo_compress_ade(sample):
    img,bpp = hific_lo_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

In [3]:
image_compression_methods = [
    jpeg_compress_ade,
    webp_compress_ade,
    mbt2018_compress_ade,
    hific_lo_compress_ade
]
models = [
    "nvidia/segformer-b0-finetuned-ade-512-512",
    # "nvidia/segformer-b2-finetuned-ade-512-512",
    # "nvidia/segformer-b5-finetuned-ade-640-640"
]

In [4]:
ade20k = [load_dataset("scene_parse_150",split='validation')]

In [5]:
for method in image_compression_methods:
    ade20k.append(ade20k[0].map(method))

In [6]:
mean_iou = evaluate.load("mean_iou")

In [7]:
from torchvision import transforms
import numpy as np
def evaluate_segmentation_miou(data):
    x = []
    y = []
    for sample in data:
        ground_truth = sample['annotation']
        img = sample['image']
        inputs = extractor(images=img, return_tensors="pt",do_resize=False).to("cuda:0")
        outputs = model(**inputs)
        logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
        predictions = logits.argmax(dim=1)[0].detach().cpu()
        resize = transforms.Resize(
            (ground_truth.height,ground_truth.width),
            antialias=True
        )
        predictions = resize(
            predictions.unsqueeze(0).unsqueeze(0)
        )[0][0]
    
        x.append(np.array(ground_truth))
        y.append(np.array(predictions))
    return mean_iou.compute(
        predictions=y,
        references=x,
        num_labels=150,
        ignore_index=255,
        reduce_labels=True
    )['mean_iou']

In [9]:
eval_results = []
for i_model,model_name in enumerate(models):
    eval_results.append([])
    extractor = AutoFeatureExtractor.from_pretrained(model_name)
    model = SegformerForSemanticSegmentation.from_pretrained(model_name).to("cuda:0")
    for i_method,method in enumerate(ade20k):
        print(i_model, ",", i_method)
        eval_results[i_model].append(
            evaluate_segmentation_miou(method)
        )

/home/dan/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/dan/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


0 , 0
0 , 1
0 , 2
0 , 3
0 , 4


In [10]:
eval_results

[[0.3189359286425029,
  0.11916789777300861,
  0.18866411196081573,
  0.20750815154022598,
  0.30084247702965283]]